## 2c. Evidence - Performance QAS Measurements

Now we collect stored, CPU and memory usage data when predicting with the model, for the Performance scenario.

### Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces.

In [1]:
import os
from mlte.session import set_context, set_store

store_path = os.path.join(os.getcwd(), "store")
os.makedirs(
    store_path, exist_ok=True
)  # Ensure we are creating the folder if it is not there.

set_context("OxfordFlower", "0.0.1")
set_store(f"local://{store_path}")

Define different folders that will be used as input or output for the data gathering process.

In [2]:
from pathlib import Path

# The path at which datasets are stored
DATASETS_DIR = Path.cwd() / "data"

# Path where the model files are stored.
MODELS_DIR = Path.cwd() / "model"

In [3]:
# This is the external script that will load and run the model for inference/prediction.
script = Path.cwd() / "model_predict.py"
args = [
    "--images",
    DATASETS_DIR,
    "--model",
    MODELS_DIR / "model_f3_a.json",
    "--weights",
    MODELS_DIR / "model_f_a.h5",
]

In [ ]:
from mlte.measurement.storage import LocalObjectSize
from mlte.value.types.integer import Integer

store_measurement = LocalObjectSize("model size")
size: Integer = store_measurement.evaluate(MODELS_DIR)
print(size)
size.save(force=True)

In [ ]:
from mlte.measurement.process_measurement import ProcessMeasurement
from mlte.measurement.cpu import LocalProcessCPUUtilization, CPUStatistics

cpu_measurement = LocalProcessCPUUtilization("predicting cpu")
cpu_stats: CPUStatistics = cpu_measurement.evaluate(
    ProcessMeasurement.start_script(script, args)
)
print(cpu_stats)
cpu_stats.save(force=True)

In [ ]:
from mlte.measurement.memory import (
    LocalProcessMemoryConsumption,
    MemoryStatistics,
)

mem_measurement = LocalProcessMemoryConsumption("predicting memory")
mem_stats: MemoryStatistics = mem_measurement.evaluate(
    ProcessMeasurement.start_script(script, args)
)
print(mem_stats)
mem_stats.save(force=True)